Segment Anything segmented Pano-image to Point Cloud

In [1]:
!pip install laspy rasterio plyfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 41.6 MB/s eta 0:00:00


In [2]:
import laspy
import numpy as np
from math import pi
import rasterio
from rasterio.enums import Resampling
from plyfile import PlyData, PlyElement

# === File paths ===
INPUT_LAS = r"/content/2025_05_22_AF_RIEGL_01.las"
OUTPUT_PLY = r"classified_result.ply"
PANO_MASK_PATH = r"labeled_mask.tif"

# === Panorama dimensions ===
MASK_WIDTH, MASK_HEIGHT = 2048, 1024
TARGET_WIDTH, TARGET_HEIGHT = 8192, 4096

def load_and_resize_mask(mask_path, target_width, target_height):
    print("📥 Loading and resizing mask...")
    try:
        with rasterio.open(mask_path) as src:
            mask = src.read(
                out_shape=(1, target_height, target_width),
                resampling=Resampling.nearest
            )[0]
        print(f"Mask shape: {mask.shape}")  # Expect (4096, 8192)
        return mask
    except Exception as e:
        print(f"❌ Error loading mask: {e}")
        exit(1)

def read_las_point_cloud(las_path):
    print("📥 Reading LAS file...")
    try:
        las = laspy.read(las_path)
        x = las.x.scaled_array()
        y = las.y.scaled_array()
        z = las.z.scaled_array()
        return las, x, y, z
    except Exception as e:
        print(f"❌ Error reading LAS file: {e}")
        exit(1)

def project_points_to_panorama(x, y, z, target_width, target_height):
    print("📏 Projecting points to panorama coordinates...")
    r = np.sqrt(x**2 + y**2 + z**2)
    r[r == 0] = 1e-6  # Avoid division by zero

    if np.any(np.isnan(r)) or np.any(np.isinf(r)):
        print("❌ Warning: NaN or inf values detected in radial distances.")
        exit(1)

    theta = np.arctan2(y, x)  # azimuth [-π, π]
    theta = (theta + 2 * pi) % (2 * pi)  # normalize [0, 2π]

    phi = np.arccos(z / r)  # zenith [0, π]

    if np.any(np.isnan(phi)) or np.any(np.isinf(phi)):
        print("❌ Warning: NaN or inf values detected in zenith angles.")
        exit(1)

    u = (theta / (2 * pi) * target_width).astype(np.int32)
    v = (phi / pi * target_height).astype(np.int32)

    print(f"u range before clip: [{u.min()}, {u.max()}]")
    print(f"v range before clip: [{v.min()}, {v.max()}]")

    u = np.clip(u, 0, target_width - 1)
    v = np.clip(v, 0, target_height - 1)

    print(f"u range after clip: [{u.min()}, {u.max()}]")
    print(f"v range after clip: [{v.min()}, {v.max()}]")

    return u, v

def sample_mask_classes(mask, u, v):
    print("🎯 Sampling mask labels for each point...")
    try:
        classes = mask[v, u]
        classes = np.clip(classes, 0, 255).astype(np.uint8)
        return classes
    except Exception as e:
        print(f"❌ Error sampling mask: {e}")
        exit(1)

def save_ply_with_class(las, x, y, z, sampled_class, output_path):
    print("💾 Saving classified PLY file...")
    try:
        # Gather LAS dimension info
        las_dims = [(dim.name, dim.dtype) for dim in las.point_format.dimensions]
        print(f"LAS dimensions: {las_dims}")

        ply_dtype_map = {
            'int8': 'i1', 'uint8': 'u1',
            'int16': 'i2', 'uint16': 'u2',
            'int32': 'i4', 'uint32': 'u4',
            'float32': 'f4', 'float64': 'f8'
        }

        # Start dtype with x,y,z as float32
        ply_dtype = [('x', 'f4'), ('y', 'f4'), ('z', 'f4')]

        # Add other fields from LAS (excluding raw X,Y,Z)
        for dim_name, dim_dtype in las_dims:
            if dim_name not in ['X', 'Y', 'Z']:
                ply_dim_name = dim_name.lower()
                ply_dim_dtype = ply_dtype_map.get(str(dim_dtype), 'f4')
                ply_dtype.append((ply_dim_name, ply_dim_dtype))

        # Add class field as uint8
        ply_dtype.append(('class', 'u1'))

        # Prepare structured array
        vertex = np.zeros(x.shape[0], dtype=ply_dtype)
        vertex['x'] = x.astype(np.float32)
        vertex['y'] = y.astype(np.float32)
        vertex['z'] = z.astype(np.float32)

        for dim_name, _ in las_dims:
            if dim_name not in ['X', 'Y', 'Z']:
                try:
                    vertex[dim_name.lower()] = las[dim_name]
                except ValueError as e:
                    print(f"⚠️ Warning: Could not copy field '{dim_name}': {e}")

        vertex['class'] = sampled_class

        ply_element = PlyElement.describe(vertex, 'vertex')
        PlyData([ply_element], text=False).write(output_path)
        print(f"✅ Done. Output saved to: {output_path}")

    except Exception as e:
        print(f"❌ Error saving PLY file: {e}")
        exit(1)

def main():
    mask = load_and_resize_mask(PANO_MASK_PATH, TARGET_WIDTH, TARGET_HEIGHT)
    las, x, y, z = read_las_point_cloud(INPUT_LAS)
    u, v = project_points_to_panorama(x, y, z, TARGET_WIDTH, TARGET_HEIGHT)
    sampled_class = sample_mask_classes(mask, u, v)
    save_ply_with_class(las, x, y, z, sampled_class, OUTPUT_PLY)

if __name__ == "__main__":
    main()


📥 Loading and resizing mask...


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Mask shape: (4096, 8192)
📥 Reading LAS file...
📏 Projecting points to panorama coordinates...
u range before clip: [0, 8191]
v range before clip: [557, 3035]
u range after clip: [0, 8191]
v range after clip: [557, 3035]
🎯 Sampling mask labels for each point...
💾 Saving classified PLY file...
LAS dimensions: [('X', dtype('int32')), ('Y', dtype('int32')), ('Z', dtype('int32')), ('intensity', dtype('uint16')), ('return_number', None), ('number_of_returns', None), ('scan_direction_flag', None), ('edge_of_flight_line', None), ('classification', None), ('synthetic', None), ('key_point', None), ('withheld', None), ('scan_angle_rank', dtype('int8')), ('user_data', dtype('uint8')), ('point_source_id', dtype('uint16')), ('gps_time', dtype('float64')), ('red', dtype('uint16')), ('green', dtype('uint16')), ('blue', dtype('uint16')), ('Amplitude', dtype('int16')), ('Reflectance', dtype('int16')), ('Deviation', dtype('int16'))]
✅ Done. Output saved to: classified_result.ply
